In [178]:

import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from mlxtend.frequent_patterns import apriori

In [179]:
data = pd.read_csv("sales.csv")
data.head()

,CustomerID,ProductID,OrderQty,OrderID
0,11000,771,1,43793
1,11000,779,1,51522
2,11000,878,1,51522
3,11000,966,1,57418
4,11000,934,1,57418


In [180]:

df = pd.DataFrame(data)

# Nhóm các ProductKey cùng SalesOrderID
data_grouped = df.groupby('OrderID')['ProductID'].apply(list)

# In kết quả
print(data_grouped)

OrderID
43659    [776, 777, 778, 771, 772, 773, 774, 714, 716, ...
43660                                           [762, 758]
43661    [745, 743, 747, 712, 715, 742, 775, 778, 711, ...
43662    [764, 770, 730, 754, 725, 762, 765, 768, 753, ...
43663                                                [760]
                               ...                        
75119                                      [921, 930, 873]
75120                                      [878, 884, 712]
75121                                      [921, 930, 707]
75122                                           [878, 712]
75123                                      [878, 879, 712]
Name: ProductID, Length: 31465, dtype: object


In [181]:
# Chuyển đổi dữ liệu group thành dạng list của list (mỗi list là một transaction)
transactions = [[str(item) for item in transaction] for transaction in data_grouped]

# Sử dụng MultiLabelBinarizer để chuyển đổi dữ liệu thành dạng mã hóa
mlb = MultiLabelBinarizer()
df_encoded = pd.DataFrame(mlb.fit_transform(transactions), columns=mlb.classes_)

# Tính toán frequent itemsets với minimum support là 0.02
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)

frequent_itemsets

c:\Users\N\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.097982,(707)
1,0.095567,(708)
2,0.098204,(711)
3,0.107485,(712)
4,0.013634,(713)
...,...,...
3401,0.010011,"(876, 715, 884, 865, 883, 864, 877, 711, 712)"
3402,0.010329,"(876, 715, 884, 870, 883, 864, 877, 711, 712)"
3403,0.010011,"(876, 715, 884, 865, 870, 864, 877, 711, 883)"
3404,0.010011,"(876, 715, 884, 865, 870, 864, 877, 883, 712)"


In [182]:
frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)
frequent_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x)) > 1]
frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(list)

C:\Users\N\AppData\Local\Temp\ipykernel_14064\2144805677.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(list)


In [183]:
frequent_itemsets

,support,itemsets
303,0.053774,"[870, 871]"
304,0.048339,"[870, 872]"
162,0.037248,"[715, 712]"
177,0.032385,"[870, 712]"
142,0.032131,"[712, 711]"
...,...,...
3022,0.010011,"[884, 711, 877, 864, 708, 880, 883]"
1362,0.010011,"[880, 864, 865, 883]"
2294,0.010011,"[715, 883, 708, 711, 880, 712]"
2296,0.010011,"[858, 859, 708, 711, 716, 712]"


In [184]:
type(frequent_itemsets.iloc[0]['itemsets'])

list

In [185]:
frequent_itemsets.to_csv("frequent_itemsets.csv", index=False)

In [186]:
load = pd.read_csv("frequent_itemsets.csv")


In [187]:
# load['itemsets'] = load['itemsets'].apply(list)
load

,support,itemsets
0,0.053774,"['870', '871']"
1,0.048339,"['870', '872']"
2,0.037248,"['715', '712']"
3,0.032385,"['870', '712']"
4,0.032131,"['712', '711']"
...,...,...
3296,0.010011,"['884', '711', '877', '864', '708', '880', '883']"
3297,0.010011,"['880', '864', '865', '883']"
3298,0.010011,"['715', '883', '708', '711', '880', '712']"
3299,0.010011,"['858', '859', '708', '711', '716', '712']"


In [188]:
import ast
def predict_next_purchase(product_id):
    with open("output.txt", "r") as file:
        spade_output_lines = file.readlines()

    # Predict next purchase from SPADE output
    spade_predictions = []
    for line in spade_output_lines:
        line = line.replace("\n", "")
        line_lst = line.split(' -1 ')
        if line_lst[0] == str(product_id):
            support = int(line_lst[-1].replace("#SUP: ", ""))
            for sequence in line_lst[1:-1]:
                spade_predictions.append([sequence, support])
    top_predictions = sorted(spade_predictions, key=lambda x: x[1], reverse=True)[:10]

    frequent_itemsets = pd.read_csv("frequent_itemsets.csv")
    # Predict combination from frequent itemsets
    ar_predictions = []
    for prediction in top_predictions:
        items = prediction[0].split()
        related_itemset = set()
        for item in items:
            max_support = 0
            max_combination = None
            for index, row in frequent_itemsets.iterrows():
                lst = ast.literal_eval(row['itemsets'])
                if item in lst and len(lst) > 1:
                    if row['support'] > max_support:
                        max_support = row['support']
                        max_combination = lst
            if max_combination:
                related_itemset.update(max_combination)  
        ar_predictions.append(list(related_itemset))

    return top_predictions, ar_predictions

In [203]:
product_id = 793  
spade_predictions, ar_predictions = predict_next_purchase(product_id)

print("Predicted next purchase from SPADE:", spade_predictions)
print("Predicted next purchases from association rules:", ar_predictions)

Predicted next purchase from SPADE: []
Predicted next purchases from association rules: []


In [190]:
import pyodbc 
import pandas as pd
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=HEO;"
                      "Database=DW_Inventory;"
                      "Trusted_Connection=yes;")

cursor = cnxn.cursor()
query = """

"""

In [191]:
product_info = """
SELECT distinct(ProductID), ProductName from Dim_Product
"""
product_info = pd.read_sql(product_info, cnxn)
product_info

C:\Users\N\AppData\Local\Temp\ipykernel_14064\4073179723.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_info = pd.read_sql(product_info, cnxn)


,ProductID,ProductName
0,1,Adjustable Race
1,2,Bearing Ball
2,3,BB Ball Bearing
3,4,Headset Ball Bearings
4,316,Blade
...,...,...
499,995,ML Bottom Bracket
500,996,HL Bottom Bracket
501,997,"Road-750 Black, 44"
502,998,"Road-750 Black, 48"


In [192]:
get_product_query = """
SELECT distinct(ProductID) from Dim_Product
"""
product_list = pd.read_sql_query(get_product_query, cnxn)
product_list

C:\Users\N\AppData\Local\Temp\ipykernel_14064\455720322.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_list = pd.read_sql_query(get_product_query, cnxn)


,ProductID
0,1
1,2
2,3
3,4
4,316
...,...
499,995
500,996
501,997
502,998


In [193]:
product_ids = []
next_sequences = []
combinations = []

for idx, row in product_list.iterrows():
    product_id = row['ProductID']
    spade_predictions, ar_predictions = predict_next_purchase(product_id)
    
    for spade_pred, ar_pred in zip(spade_predictions, ar_predictions):
        product_ids.append(product_id)
        next_sequences.append(spade_pred[0])
        combinations.append(ar_pred)

new_df = pd.DataFrame({
    "productID": product_ids,
    "next_sequence": next_sequences,
    "combination_from_next_sequence": combinations
})
new_df

,productID,next_sequence,combination_from_next_sequence
0,707,712,"[715, 712]"
1,707,711,"[711, 712]"
2,707,707,"[707, 712]"
3,707,708,"[708, 712]"
4,707,715,"[715, 712]"
...,...,...,...
84,877,870,"[870, 871]"
85,877,712,"[715, 712]"
86,880,712,"[715, 712]"
87,921,873,"[921, 873]"


In [194]:
CREATE_PREDICTION_TABLE_IF_NOT_EXIST_QUERY = """
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'DimPrediction')
BEGIN
CREATE TABLE DimPrediction(
    ProductID INT,
    NextSequence VARCHAR(255),
    CombinationFromNextSequence VARCHAR(255)
);
END
"""
INSERT_PREDICTION_QUERY = """
INSERT INTO DimPrediction(ProductID, NextSequence, CombinationFromNextSequence)
VALUES (?, ?, ?)
"""


In [195]:
cursor = cnxn.cursor()
cursor.execute(CREATE_PREDICTION_TABLE_IF_NOT_EXIST_QUERY)
cnxn.commit()

In [199]:
cursor = cnxn.cursor()
for idx, row in df.iterrows():
    cursor.execute(INSERT_PREDICTION_QUERY, row['productID'], str(row['next_sequence']), str(row['combination_from_next_sequence']))
    cnxn.commit()

In [197]:
new_df['combination_from_next_sequence'].tolist()

[['715', '712'],
 ['711', '712'],
 ['707', '712'],
 ['708', '712'],
 ['715', '712'],
 ['870', '871'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['711', '712'],
 ['707', '712'],
 ['708', '712'],
 ['870', '871'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['711', '712'],
 ['708', '712'],
 ['707', '712'],
 ['715', '712'],
 ['870', '871'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['711', '712'],
 ['708', '712'],
 ['707', '712'],
 ['715', '712'],
 ['715', '712'],
 ['870', '871'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['711', '712'],
 ['715', '712'],
 ['708', '712'],
 ['715', '712'],
 ['715', '712'],
 ['707', '712'],
 ['714', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'],
 ['711', '712'],
 ['715', '712'],
 ['715', '712'],
 ['715', '712'

In [198]:
df = new_df

def replace_product_names(row):
    product_ids = row["next_sequence"].split()  # Split the string into a list of product IDs
    product_names = [product_info.loc[product_info["ProductID"] == int(product_id), "ProductName"].values[0] for product_id in product_ids]
    return ", ".join(product_names)

# Apply the function to replace product IDs with product names for the "next_sequence" column
df["next_sequence"] = df.apply(replace_product_names, axis=1)

# Function to replace product IDs with product names in the list
def replace_combination_names(combination_list):
    return [product_info.loc[product_info["ProductID"] == int(product_id), "ProductName"].values[0] for product_id in combination_list]

# Apply the function to replace product IDs with product names for the "combination_from_next_sequence" column
df["combination_from_next_sequence"] = df["combination_from_next_sequence"].apply(replace_combination_names)

df

,productID,next_sequence,combination_from_next_sequence
0,707,AWC Logo Cap,"[Long-Sleeve Logo Jersey, L, AWC Logo Cap]"
1,707,"Sport-100 Helmet, Blue","[Sport-100 Helmet, Blue, AWC Logo Cap]"
2,707,"Sport-100 Helmet, Red","[Sport-100 Helmet, Red, AWC Logo Cap]"
3,707,"Sport-100 Helmet, Black","[Sport-100 Helmet, Black, AWC Logo Cap]"
4,707,"Long-Sleeve Logo Jersey, L","[Long-Sleeve Logo Jersey, L, AWC Logo Cap]"
...,...,...,...
84,877,Water Bottle - 30 oz.,"[Water Bottle - 30 oz., Mountain Bottle Cage]"
85,877,AWC Logo Cap,"[Long-Sleeve Logo Jersey, L, AWC Logo Cap]"
86,880,AWC Logo Cap,"[Long-Sleeve Logo Jersey, L, AWC Logo Cap]"
87,921,Patch Kit/8 Patches,"[Mountain Tire Tube, Patch Kit/8 Patches]"


In [201]:
df.to_excel('blablabl.xlsx', index=False)